In [3]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [4]:
df = 'data/global air pollution dataset.csv'
air = pd.read_csv(df)

In [5]:
air.head()

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good
2,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate
3,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good
4,France,Punaauia,22,Good,0,Good,22,Good,0,Good,6,Good


In [6]:
air.nunique()

Country                 175
City                  23462
AQI Value               347
AQI Category              6
CO AQI Value             34
CO AQI Category           3
Ozone AQI Value         213
Ozone AQI Category        5
NO2 AQI Value            59
NO2 AQI Category          2
PM2.5 AQI Value         383
PM2.5 AQI Category        6
dtype: int64

In [7]:
air.isnull().sum()

Country               427
City                    1
AQI Value               0
AQI Category            0
CO AQI Value            0
CO AQI Category         0
Ozone AQI Value         0
Ozone AQI Category      0
NO2 AQI Value           0
NO2 AQI Category        0
PM2.5 AQI Value         0
PM2.5 AQI Category      0
dtype: int64

In [8]:
air["Country"].value_counts()

Country
United States of America    2872
India                       2488
Brazil                      1562
Germany                     1345
Russian Federation          1241
                            ... 
Vanuatu                        1
Solomon Islands                1
Republic of Korea              1
Saint Lucia                    1
Monaco                         1
Name: count, Length: 175, dtype: int64

In [9]:
# dropping 'PM2.5 Value' parameters since it is highly correlated with 'AQI Value' with correlation = 0.98
air.drop(columns=['PM2.5 AQI Category', 'City','PM2.5 AQI Value'],inplace=True)

In [10]:
air.head()

,Country,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category
0,Russian Federation,51,Moderate,1,Good,36,Good,0,Good
1,Brazil,41,Good,1,Good,5,Good,1,Good
2,Italy,66,Moderate,1,Good,39,Good,2,Good
3,Poland,34,Good,1,Good,34,Good,0,Good
4,France,22,Good,0,Good,22,Good,0,Good


In [11]:
air.shape

(23463, 9)

In [12]:
air.dropna(inplace=True)

In [13]:
air.shape

(23036, 9)

In [14]:
air.isnull().sum()

Country               0
AQI Value             0
AQI Category          0
CO AQI Value          0
CO AQI Category       0
Ozone AQI Value       0
Ozone AQI Category    0
NO2 AQI Value         0
NO2 AQI Category      0
dtype: int64

In [15]:
# Assuming 'air' is your DataFrame
# Step 1: Count occurrences of each country
country_counts = air['Country'].value_counts()

# Step 2: Identify countries with a count below 50
other_countries = country_counts[country_counts < 50].index

# Step 3: Replace countries with less than 50 occurrences with 'Others'
air['Country'] = air['Country'].apply(lambda x: 'Others' if x in other_countries else x)

# Step 4: Group by 'Country' and count occurrences
grouped_data = air.groupby('Country').size().reset_index(name='Count')

grouped_data

,Country,Count
0,Argentina,68
1,Australia,85
2,Austria,68
3,Bangladesh,80
4,Belgium,284
...,...,...
58,United Republic of Tanzania,229
59,United States of America,2872
60,Uzbekistan,58
61,Venezuela (Bolivarian Republic of),90


In [16]:
air['Country'].value_counts()

Country
United States of America    2872
India                       2488
Others                      1926
Brazil                      1562
Germany                     1345
                            ... 
Denmark                       60
Kenya                         60
Myanmar                       59
Ghana                         58
Uzbekistan                    58
Name: count, Length: 63, dtype: int64

In [17]:
X = air.drop(columns=['AQI Value'],axis=1)

In [18]:
X.head()

,Country,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category
0,Russian Federation,Moderate,1,Good,36,Good,0,Good
1,Brazil,Good,1,Good,5,Good,1,Good
2,Italy,Moderate,1,Good,39,Good,2,Good
3,Poland,Good,1,Good,34,Good,0,Good
4,France,Good,0,Good,22,Good,0,Good


In [19]:
y = air['AQI Value']

In [20]:
y.head()

0    51
1    41
2    66
3    34
4    22
Name: AQI Value, dtype: int64

In [21]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [22]:
ordinal_categories = ['Good', 'Moderate', 'Unhealthy for Sensitive Groups', 'Unhealthy', 'Very Unhealthy', 'Hazardous']
ordinal_features = ['CO AQI Category', 'Ozone AQI Category', 'NO2 AQI Category', 'AQI Category']

preprocessor = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['CO AQI Value', 'Ozone AQI Value', 'NO2 AQI Value']),
    ('tnf2', OrdinalEncoder(categories=[ordinal_categories] * len(ordinal_features), dtype=int), ['CO AQI Category', 'Ozone AQI Category', 'NO2 AQI Category', 'AQI Category']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first', dtype=int), ['Country'])
], remainder='passthrough')

In [23]:
X = preprocessor.fit_transform(X)

In [24]:
X.shape

(23036, 69)

In [25]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((18428, 69), (4608, 69))

In [26]:
X_train

array([[ 2.,  4.,  3., ...,  0.,  0.,  0.],
       [ 0., 24.,  0., ...,  0.,  0.,  0.],
       [ 0., 38.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 5., 14.,  3., ...,  0.,  0.,  0.],
       [ 1., 14.,  3., ...,  0.,  0.,  0.],
       [ 1., 61.,  1., ...,  0.,  0.,  0.]])

In [27]:
X_train.shape

(18428, 69)

In [28]:
X_test

array([[ 2., 23.,  0., ...,  0.,  0.,  0.],
       [ 1., 43.,  0., ...,  0.,  0.,  0.],
       [ 0., 37.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 4., 41.,  4., ...,  0.,  0.,  0.],
       [ 1.,  5.,  4., ...,  0.,  0.,  0.],
       [ 1., 39.,  0., ...,  0.,  0.,  0.]])

In [29]:
X_test.shape

(4608, 69)

In [30]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [31]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 21.3918
- Mean Absolute Error: 12.7616
- R2 Score: 0.8553
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 22.1203
- Mean Absolute Error: 12.8885
- R2 Score: 0.8486


Lasso
Model performance for Training set
- Root Mean Squared Error: 21.9566
- Mean Absolute Error: 13.1341
- R2 Score: 0.8475
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 22.6743
- Mean Absolute Error: 13.2162
- R2 Score: 0.8409


Ridge
Model performance for Training set
- Root Mean Squared Error: 21.3920
- Mean Absolute Error: 12.7596
- R2 Score: 0.8553
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 22.1191
- Mean Absolute Error: 12.8852
- R2 Score: 0.8486


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 14.4850
- Mean Absolute Error: 7.4758
- R2 Score: 0.9336
----------

In [32]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
7,CatBoosting Regressor,0.957156
5,Random Forest Regressor,0.955545
6,XGBRegressor,0.955519
8,AdaBoost Regressor,0.943583
4,Decision Tree,0.941941
3,K-Neighbors Regressor,0.892088
2,Ridge,0.848644
0,Linear Regression,0.848628
1,Lasso,0.840950


In [33]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import numpy as np

# Iterate through each model in your list or dictionary
for i, model_name in enumerate(models.keys()):
    # Get the model
    model = models[model_name]
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on both training and test sets
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate performance metrics
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Cross-validation on the training data
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    cv_mean = np.mean(cv_scores)
    cv_std = np.std(cv_scores)
    
    # Print out results
    print(f"Model: {model_name}")
    print(f"Training MSE: {train_mse:.4f}, Testing MSE: {test_mse:.4f}")
    print(f"Training R²: {train_r2:.4f}, Testing R²: {test_r2:.4f}")
    print(f"Cross-Validation R² (Mean ± Std): {cv_mean:.4f} ± {cv_std:.4f}")
    print("="*40)
    
    # Check for overfitting
    if train_mse < test_mse and train_r2 > test_r2:
        print("Possible overfitting detected for this model.\n")
    else:
        print("Good to go!")


Model: Linear Regression
Training MSE: 457.6070, Testing MSE: 489.3060
Training R²: 0.8553, Testing R²: 0.8486
Cross-Validation R² (Mean ± Std): -37621898979463024.0000 ± 75243797958926032.0000
Possible overfitting detected for this model.

Model: Lasso
Training MSE: 482.0915, Testing MSE: 514.1253
Training R²: 0.8475, Testing R²: 0.8409
Cross-Validation R² (Mean ± Std): 0.8475 ± 0.0095
Possible overfitting detected for this model.

Model: Ridge
Training MSE: 457.6162, Testing MSE: 489.2538
Training R²: 0.8553, Testing R²: 0.8486
Cross-Validation R² (Mean ± Std): 0.8547 ± 0.0086
Possible overfitting detected for this model.

Model: K-Neighbors Regressor
Training MSE: 209.8149, Testing MSE: 348.8217
Training R²: 0.9336, Testing R²: 0.8921
Cross-Validation R² (Mean ± Std): 0.8897 ± 0.0102
Possible overfitting detected for this model.

Model: Decision Tree
Training MSE: 33.2414, Testing MSE: 187.7631
Training R²: 0.9895, Testing R²: 0.9419
Cross-Validation R² (Mean ± Std): 0.9438 ± 0.0079